In [3]:
import pandas as pd

df =pd.read_csv('slot_data_マルハン石和店_20240514.csv')
train = df[['台番号','差枚','date']]
train.info()
train['差枚'] = train['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
train['date'] = pd.to_datetime(train['date'])
train['date_num'] = train['date'].dt.strftime('%Y%m%d').astype(str)
train['month'] = train['date'].dt.month.astype(int)
train['date_tens_digit'] = train['date_num'].str[-2].astype(int)
train['date_units_digit'] = train['date_num'].str[-1].astype(int)
train['slot_num_units_digit'] = train['台番号'].astype(str).str[-1].astype(int)
train['win_lose'] = train['差枚'].apply(lambda x: 1 if x > 0 else 0)
train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89042 entries, 0 to 89041
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   台番号     89042 non-null  int64 
 1   差枚      89042 non-null  object
 2   date    89042 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


C:\Users\文傑\AppData\Local\Temp\ipykernel_7728\2207776606.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['差枚'] = train['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
C:\Users\文傑\AppData\Local\Temp\ipykernel_7728\2207776606.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['差枚'] = train['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
C:\Users\文傑\AppData\Local\Temp\ipykernel_7728\2207776606.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,台番号,差枚,date,date_num,month,date_tens_digit,date_units_digit,slot_num_units_digit,win_lose
0,682,2500,2024-05-14,20240514,5,1,4,2,1
1,683,-700,2024-05-14,20240514,5,1,4,3,0
2,684,-500,2024-05-14,20240514,5,1,4,4,0
3,685,600,2024-05-14,20240514,5,1,4,5,1
4,686,-700,2024-05-14,20240514,5,1,4,6,0
...,...,...,...,...,...,...,...,...,...
89037,670,-500,2023-04-15,20230415,4,1,5,0,0
89038,671,1200,2023-04-15,20230415,4,1,5,1,1
89039,672,100,2023-04-15,20230415,4,1,5,2,1
89040,673,-2400,2023-04-15,20230415,4,1,5,3,0


In [3]:
df.columns

Index(['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', '合成確率', 'BB確率', 'RB確率', 'date'], dtype='object')

In [4]:
train = train[train['date_units_digit'] == 5]
train

,台番号,差枚,date,date_num,month,date_tens_digit,date_units_digit,slot_num_units_digit,win_lose
2178,682,2700,2024-05-05,20240505,5,0,5,2,1
2179,683,1100,2024-05-05,20240505,5,0,5,3,1
2180,684,-1000,2024-05-05,20240505,5,0,5,4,0
2181,685,-900,2024-05-05,20240505,5,0,5,5,0
2182,686,-200,2024-05-05,20240505,5,0,5,6,0
...,...,...,...,...,...,...,...,...,...
89037,670,-500,2023-04-15,20230415,4,1,5,0,0
89038,671,1200,2023-04-15,20230415,4,1,5,1,1
89039,672,100,2023-04-15,20230415,4,1,5,2,1
89040,673,-2400,2023-04-15,20230415,4,1,5,3,0


In [5]:
#尾數等於奇數
# train = train[train['date_units_digit'] % 2 != 0]

In [90]:
#用月份分組 看差枚的總數
train.groupby('date_units_digit')['差枚'].sum()

date_units_digit
0    -685400
1    -112900
2    -750300
3    -759000
4    -776700
5     580000
6    -554600
7     -50600
8    -134900
9   -1224500
Name: 差枚, dtype: int32

機器學習

In [79]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , mean_squared_error , r2_score , confusion_matrix , f1_score
import warnings
warnings.filterwarnings("ignore")

In [80]:
#predict [win_lose] = 1 or 0
y = train['win_lose']
x = train.drop(['差枚','date','date_num','win_lose','date_tens_digit'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x)
print(y)
print(x_train)
print(y_train)


       台番号  month  date_units_digit  slot_num_units_digit
2178   682      5                 5                     2
2179   683      5                 5                     3
2180   684      5                 5                     4
2181   685      5                 5                     5
2182   686      5                 5                     6
...    ...    ...               ...                   ...
89037  670      4                 5                     0
89038  671      4                 5                     1
89039  672      4                 5                     2
89040  673      4                 5                     3
89041  674      4                 5                     4

[8754 rows x 4 columns]
2178     1
2179     1
2180     0
2181     0
2182     0
        ..
89037    0
89038    1
89039    1
89040    0
89041    1
Name: win_lose, Length: 8754, dtype: int64
       台番号  month  date_units_digit  slot_num_units_digit
34256  574     12                 5                     4

In [81]:


lg = LogisticRegression()
svm = SVC(probability= True , random_state = 42 , kernel = 'linear')
nb = GaussianNB()
dt = DecisionTreeClassifier(random_state = 42)
rf = RandomForestClassifier(random_state = 100)
ada = AdaBoostClassifier()
per =  Perceptron()

lg.fit(x_train,y_train)
y_pred_lg = lg.predict(x_test)
print(f'valid score of logistic:{accuracy_score(y_test,y_pred_lg)}')
nb.fit(x_train,y_train)
y_pred_nb = nb.predict(x_test)
print(f'valid score of naive bayes:{accuracy_score(y_test,y_pred_nb)}')
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_test)
print(f'valid score of DecisionTree:{accuracy_score(y_test,y_pred_dt)}')
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)
print(f'valid score of RandomForest:{accuracy_score(y_test,y_pred_rf)}')
ada.fit(x_train,y_train)
y_pred_ada = ada.predict(x_test)
print(f'valid score of AdaBoost:{accuracy_score(y_test,y_pred_ada)}')
per.fit(x_train,y_train)
y_pred_per = per.predict(x_test)
print(f'valid score of Perceptron:{accuracy_score(y_test,y_pred_per)}')
# svm.fit(x_train,y_train)
# y_pred_svm = svm.predict(x_test)
# print(f'valid score of SVM:{accuracy_score(y_test,y_pred_svm)}')

valid score of logistic:0.6036550542547116
valid score of naive bayes:0.6036550542547116
valid score of DecisionTree:0.5522558537978298
valid score of RandomForest:0.5465448315248429
valid score of AdaBoost:0.6065105653912051
valid score of Perceptron:0.6036550542547116


In [86]:
print(f'Label:\n{y_test.groupby(by = lambda x: y_test[x]).count()}\n'+'-'*100)
print(f'confusion matrix of logistic:\n{confusion_matrix(y_test,y_pred_lg)}\n'+'-'*100)
# print(f'confusion matrix of SVM:\n{confusion_matrix(y_test,y_pred_svm)}\n'+'-'*100)
print(f'confusion matrix of naive bayes:\n{confusion_matrix(y_test,y_pred_nb)}\n'+'-'*100)
print(f'confusion matrix of DecisionTree:\n{confusion_matrix(y_test,y_pred_dt)}\n'+'-'*100)
print(f'confusion matrix of RandomForest:\n{confusion_matrix(y_test,y_pred_rf)}\n'+'-'*100)
print(f'confusion matrix of AdaBoost:\n{confusion_matrix(y_test,y_pred_ada)}\n'+'-'*100)
print(f'confusion matrix of Perceptron:\n{confusion_matrix(y_test,y_pred_per)}\n'+'-'*100)

Label:
0    1057
1     694
Name: win_lose, dtype: int64
----------------------------------------------------------------------------------------------------
confusion matrix of logistic:
[[1057    0]
 [ 694    0]]
----------------------------------------------------------------------------------------------------
confusion matrix of naive bayes:
[[1057    0]
 [ 694    0]]
----------------------------------------------------------------------------------------------------
confusion matrix of DecisionTree:
[[791 266]
 [518 176]]
----------------------------------------------------------------------------------------------------
confusion matrix of RandomForest:
[[685 372]
 [422 272]]
----------------------------------------------------------------------------------------------------
confusion matrix of AdaBoost:
[[1049    8]
 [ 681   13]]
----------------------------------------------------------------------------------------------------
confusion matrix of Perceptron:
[[1057    0]
 [ 69

In [83]:
print(f'precision of logistic:')

precision of logistic:


In [84]:
from sklearn.metrics import recall_score
print(f'valid recall score of logistic:{recall_score(y_test,y_pred_lg)}')
# print(f'valid recall score of SVM:{recall_score(y_valid,y_pred_svm)}')
print(f'valid recall score of naive bayes:{recall_score(y_test,y_pred_nb)}')
print(f'valid recall score of DecisionTree:{recall_score(y_test,y_pred_dt)}')
print(f'valid recall score of RandomForest:{recall_score(y_test,y_pred_rf)}')
print(f'valid recall score of AdaBoost:{recall_score(y_test,y_pred_ada)}')
print(f'valid recall score of Perceptron:{recall_score(y_test,y_pred_per)}')

valid recall score of logistic:0.0
valid recall score of naive bayes:0.0
valid recall score of DecisionTree:0.25360230547550433
valid recall score of RandomForest:0.3919308357348703
valid recall score of AdaBoost:0.018731988472622477
valid recall score of Perceptron:0.0


In [85]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
# reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)

# models, predictions = reg.fit(x_train, x_test, y_train, y_test)
# print(models)
